In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
from urls import data_urls
from darts import TimeSeries
import torch

In [2]:
dados_treino = pd.read_csv(data_urls['dados_treino'])
dados_teste =  pd.read_csv(data_urls['dados_teste'])
feriados = pd.read_csv(data_urls['feriados'])
preco_petroleo = pd.read_csv(data_urls['preco_petroleo'])
lojas = pd.read_csv(data_urls['lojas'])
transacoes = pd.read_csv(data_urls['transacoes'])


# Introdução

As técnicas de previsão de séries temporais assumem uma importância crítica no panorama empresarial contemporâneo, dada a sua capacidade de antecipar tendências futuras e comportamentos de mercado com base na análise de dados históricos. Em um ambiente de negócios caracterizado por volatilidade, incerteza, complexidade e ambiguidade, a habilidade de prever com precisão eventos futuros torna-se um diferencial competitivo fundamental. Empresas que dominam essas técnicas podem otimizar a gestão de estoques, aprimorar a alocação de recursos, antecipar mudanças na demanda dos consumidores e ajustar estratégias de marketing de acordo com projeções futuras. Além disso, a previsão de séries temporais é vital para a tomada de decisões financeiras estratégicas, como investimentos e orçamentos, permitindo que as organizações se posicionem de maneira proativa frente às oportunidades e desafios do mercado. Portanto, a adoção dessas técnicas representa não apenas uma ferramenta para a maximização da eficiência operacional, mas também um elemento estratégico para a sustentabilidade e crescimento a longo prazo no cenário empresarial global.

Neste artigo exploraremos técnicas de previsão de séries temporais utilizando algorítimos de machine learning, passando por todas as etapas do processo, desde a limpeza e preparação dos dados, até a criação e validação dos modelos criados para prever as futuras tendências para as vendas da cadeia de lojas. Para isso foi utilizado um dataset das vendas  de produtos de uma rede de lojas do equador entre os anos de 2013 e 2017. 





# Visualizando os dados

**Fazer algumas visualizações**


## Produtos por categoria da loja 1

In [3]:
for str in dados_treino['store_nbr'].unique():
    produtos_loja1 = dados_treino.loc[dados_treino['store_nbr']== str]
    break
#produtos_loja1['date'] = pd.to_datetime(produtos_loja1['date'])
#sns.set_palette('Accent')
#sns.set_style('darkgrid')
#bx = sns.lineplot(data=produtos_loja1, x='date', y='sales',hue='family')
#bx.figure.set_size_inches(14,8)

## Adicionando a cotação do petróleo ao dataset

O conjunto dos dados representa os registros de venda de uma rede de lojas do Equador durante um período. A econômia do equador extremamente dependente do petróleo, portanto é interesante apresentar isso ao nosso modelo para que ele consiga detectar qualquer correlação na variação do preço e das vendas.

Para isso foi necessário fazer alguns tratamentos nos dados relacionados ao preço do petróleo.

**Falar sobre covariantes aqui**



#### Tratando dados faltantes para covariante

In [4]:
# Criando um dataset com todas as datas em que queremos análisar as vendas 

total_datas = np.concatenate((dados_treino['date'].unique(),dados_teste['date'].unique()))
nanobj = np.array([np.nan for d in total_datas])
dicio = {'date': total_datas,
        'preco': nanobj }
preco_petroleo_todos = pd.DataFrame(data=dicio)

#Adicionando a cotação do petroleo para todas as datas disponíveis 

for i in preco_petroleo.itertuples():
   for j in preco_petroleo_todos.itertuples():
        if i.date == j.date:
            preco_petroleo_todos.at[j.Index,'preco'] = i.dcoilwtico
            break

#Tratando dias em que a cotação do petroleo não foi registrada.
#um dia com a cotação de petróleo não registrada recebe o valor do próximo dia em que a cotação foi registrada
#Essa forma de tratamento é interessante pois preserva o comportamento do preço durante a série temporal

for i in preco_petroleo_todos.itertuples():
    if np.isnan(i.preco) == True:
        for j in preco_petroleo_todos[i.Index+1:].itertuples():
            if np.isnan(j.preco) == False:
                preco_petroleo_todos.at[i.Index,'preco'] = j.preco
                break
 

In [5]:
# sns.lineplot(data = preco_petroleo_todos, x= 'date', y = 'preco')

In [6]:
#Criando dicionário com cotação diária e criando colunas nos datasets de teste e treino

dicio_petroleo = {}
for i in preco_petroleo_todos.itertuples():
    dicio_petroleo[i.date] = i.preco

petroleo_treino = []
for i in dados_treino.itertuples():
    petroleo_treino.append(dicio_petroleo[i.date])
    
petroleo_teste = []
for i in dados_teste.itertuples():
    petroleo_teste.append(dicio_petroleo[i.date])
    
complete_data_treino = dados_treino
complete_data_treino['Petroleo'] = petroleo_treino

complete_data_teste = dados_teste
complete_data_teste['Petroleo'] = petroleo_teste


In [10]:
petroleo_prices = preco_petroleo_todos[['date','preco']]
petroleo_prices.set_index('date')
petroleo_prices_timesries = TimeSeries.from_dataframe(df = petroleo_prices,
                            time_col='date',
                            value_cols='preco',
                            fill_missing_dates = True
                         )

## Adicionando metadados das lojas 

**Falar sobre covariantes estáticas aqui**

In [11]:
complete_data_treino = pd.merge(complete_data_treino,lojas,how='inner',on = 'store_nbr')
complete_data_teste = pd.merge(complete_data_teste,lojas,how='inner',on = 'store_nbr')

In [12]:
complete_data_treino.head()

,id,date,store_nbr,family,sales,onpromotion,Petroleo,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Quito,Pichincha,D,13


In [13]:
from sklearn.preprocessing import OrdinalEncoder

complete_data_encoded_treino = complete_data_treino
oe  = OrdinalEncoder()

family = oe.fit_transform(complete_data_treino[['family']])
city = oe.fit_transform(complete_data_treino[['city']])
state = oe.fit_transform(complete_data_treino[['state']])
types = oe.fit_transform(complete_data_treino[['type']])
cluster = oe.fit_transform(complete_data_treino[['cluster']])

complete_data_encoded_treino[['family']] = family
complete_data_encoded_treino[['city']] = city
complete_data_encoded_treino[['state']] = state
complete_data_encoded_treino[['type']] = types
complete_data_encoded_treino[['cluster']] = cluster


In [14]:
complete_data_encoded_treino[['family','city','state','type','cluster']].head()

,family,city,state,type,cluster
0,0.0,18.0,12.0,3.0,12.0
1,1.0,18.0,12.0,3.0,12.0
2,2.0,18.0,12.0,3.0,12.0
3,3.0,18.0,12.0,3.0,12.0
4,4.0,18.0,12.0,3.0,12.0


## Adicionando sazonalidade semanal

# Organizando a sequência das séries



In [15]:

multi_series_data = []
for str in complete_data_encoded_treino['store_nbr'].unique():
    for f in complete_data_encoded_treino['family'].unique():
        serie = complete_data_treino.loc[(complete_data_treino['store_nbr']== str) & (complete_data_treino['family']== f)]
        serie['date'] = pd.to_datetime(serie['date'])
        multi_series_data.append(serie)
    

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

...

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['date'] = pd.to_datetime(serie['date'])
C:\Users\Felipe\AppData\Local\Temp\ipykernel_7576\3986689722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [16]:


multi_serie_TimeSeries = []
for serie in multi_series_data:
    keys = serie.iloc[0]
    dataf_static = {'family':[keys.family],
                   'city':[keys.city],
                   'state':[keys.state],
                   'type': [keys.type],
                   'cluster':[keys.cluster]}
    
    static_cov = pd.DataFrame(data=dataf_static)
    
    new = TimeSeries.from_dataframe(df=serie,
                                    time_col='date',
                                    value_cols='sales',
                                    static_covariates=static_cov,
                                    fill_missing_dates=True,
                                    fillna_value = 0)
    
    multi_serie_TimeSeries.append(new)
    
    

In [40]:
multi_series_data[0]

,id,date,store_nbr,family,sales,onpromotion,Petroleo,city,state,type,cluster
0,0,2013-01-01,1,0.0,0.0,0,93.14,18.0,12.0,3.0,12.0
33,1782,2013-01-02,1,0.0,2.0,0,93.14,18.0,12.0,3.0,12.0
66,3564,2013-01-03,1,0.0,3.0,0,92.97,18.0,12.0,3.0,12.0
99,5346,2013-01-04,1,0.0,3.0,0,93.12,18.0,12.0,3.0,12.0
132,7128,2013-01-05,1,0.0,5.0,0,93.20,18.0,12.0,3.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
55407,2991978,2017-08-11,1,0.0,1.0,0,48.81,18.0,12.0,3.0,12.0
55440,2993760,2017-08-12,1,0.0,6.0,0,47.59,18.0,12.0,3.0,12.0
55473,2995542,2017-08-13,1,0.0,1.0,0,47.59,18.0,12.0,3.0,12.0
55506,2997324,2017-08-14,1,0.0,1.0,0,47.59,18.0,12.0,3.0,12.0


In [36]:
multi_serie_train= []
multi_serie_test = []
serie_cov_price = []

for serie in multi_serie_TimeSeries:
    train = serie[:len(serie)-15]
    train = train.add_datetime_attribute(attribute = 'weekday',one_hot=True)
    cov_l =  petroleo_prices_timesries[:len(serie)]
    multi_serie_train.append(train)
    serie_cov_price.append(cov_l)
    
    test = serie[len(serie)-15:]
    test = test.add_datetime_attribute(attribute = 'weekday',one_hot=True)
    multi_serie_test.append(test)

# RNN Model

In [44]:
from darts.models import RNNModel

future_cov = serie_cov_price

RNNmodel = RNNModel(input_chunk_length = 14,
                   model = 'LSTM',
                   hidden_dim =28,
                   n_rnn_layers = 1,
                    training_length = 28,
                    loss_fn = torch.nn.MSELoss(),
                    optimizer_kwargs= {'lr':1e-3}
                   )

RNNmodel.fit(multi_serie_train[:1], future_covariates=future_cov[:1])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 3.5 K 
4 | V             | Linear           | 29    
---------------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\Felipe\anaconda3\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


RNNModel(model=LSTM, hidden_dim=28, n_rnn_layers=1, dropout=0.0, training_length=28, input_chunk_length=14, loss_fn=MSELoss(), optimizer_kwargs={'lr': 0.001})

# Testes e resultados 
**Scorre das predições**
**Visualização das predições**

In [42]:
pred = RNNmodel.predict(series=multi_serie_train[0],
                        future_covariates=future_cov[0],
                        n=15)
pred.values()


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])